In [25]:
import random

In [26]:
def generate_string_01(length,max_consecutive_zeros):
    string=''
    consecutive_zeros=0
    for _ in range(length):
        bit=random.choice('01')
        if bit=='0':
            consecutive_zeros+=1
            if consecutive_zeros>max_consecutive_zeros:
                bit='1'
                consecutive_zeros=0
        else:
            consecutive_zeros=0
        
        
        string+=bit
    return string

In [27]:
#ocena je fitnes samo
def calculate_fitness(rating): 
    return rating

In [28]:
def create_first_gen(population_size,string_length,max_consecutive_zeros):
    return [generate_string_01(string_length,max_consecutive_zeros) for _ in range(population_size)]

In [29]:
def crossover(parent1,parent2):
    random_pos=random.randint(1,len(parent1)-1)

    child1=parent1[:random_pos]+parent2[random_pos:]
    child2=parent2[:random_pos]+parent2[random_pos:]

    return child1,child2

    

In [30]:
def mutate(individual,mutation_rate,max_consecutive_zeros):
    mutated=''
    consecutive_zeros=0
    for bit in individual:
        if random.random() < mutation_rate:
            #ako je TRENUTNI bit 0 i ima vise 0 za redom nego max,menjamo ga u 1
            if bit=='0' and consecutive_zeros>max_consecutive_zeros:
                bit='1'
                mutated+=bit
                #moramo da resetujemo brojac uzastopnih sad jer nam je sad 1,pa da krene ispocetka od sledeceg
                consecutive_zeros=0  
            else: 
     
                mutated += '1' if bit == '0' else '0'
                consecutive_zeros = consecutive_zeros + 1 if bit == '1' else 0
       
        else:
            mutated+=bit
            if bit=='1':
                consecutive_zeros=0
            else: 
                consecutive_zeros+=1
    return mutated
                

In [31]:

#biramo random roditelje iz populacije,roulette
def select_parents(population,fitness_scores):
    total_fitness=sum(fitness_scores)
    selection_probabilities=[score/total_fitness for score in fitness_scores]
    #ovdje bira k=2 iz population koje ce da vrati kao roditelje,na osnovu skora koji je dobio
    #selected_indices=[parent1,parent2]
    selected_indices=random.choices(range(len(population)),weights=selection_probabilities,k=2)
    return population[selected_indices[0]],population[selected_indices[1]]
    

In [32]:
def genetic_algorithm(string_length,population_size,mutation_rate,max_consecutive_zeros):   
    #ovde ga samo inicijalizuje
    population=create_first_gen(population_size,string_length,max_consecutive_zeros)
   
    #mapa k=jedinka v=ocena
    population_fitness={individual : 0 for individual in population}

    overall_best_string = None
    overall_best_fitness = 0

    
    best_string= None
    best_fitness=0
    second_best_string=None
    second_best_fitness=0

    #dok ga ne prekine user da ide
    while True:
        for individual in population_fitness:
            population_fitness[individual]=calculate_fitness(int(input(f"Rate {individual}")))

             #ovde da nadjem overall najbolji,>= jer zelim da cuvam najnovijeg
            if population_fitness[individual] >= overall_best_fitness:
                overall_best_string = individual
                overall_best_fitness = population_fitness[individual]
    
        #uzima prva dva jer je silazno sortirano max..min tu mi je bila greska sad cuva najbolje ovde,
        #a dole kad pravim novu generaciju sortiram od najgoreg do najboljeg da mi lakse da skinem najgora 2
        #i stavim tu decu,ovde se svakako opet sortira pa ne brinem dalje o redosledu
          
        sorted_population=sorted(population_fitness.items(),key=lambda item: item[1],reverse=True)
        best_string, best_fitness = sorted_population[0]
        second_best_string, second_best_fitness = sorted_population[1]

        #ovde pita da l zeli da nastavi,da ide u iducu generaciju
        user_input = input(f"Best string: {best_string}, Best fitness: {best_fitness}\n"
                           f"Second best string: {second_best_string}, Second best fitness: {second_best_fitness}\n"
                           f"Overall best string: {overall_best_string},Overall best fitness: {overall_best_fitness}\n"
                           "Continue to the next generation? (y/n): ")
        if user_input.lower()!='y':
                break

        #dalje biramo roditelje za sledecu gen
        parents=select_parents(population,[fitness for _, fitness in sorted_population])
       
        child1,child2=crossover(parents[0],parents[1])

        child1 = mutate(child1,mutation_rate,max_consecutive_zeros)
        child2 = mutate(child2,mutation_rate,max_consecutive_zeros)

        #zamijeni sve osim dva najbolja iz prosle generacije,oni su na kraju 
       #population=[child1,child2]+population[:2]  ovo nije bilo dobro zato sam imala ruzne rezultate
       #cak sam i pogresno stavila treba 2: hahahahah
        #ovde sad znaci mice 2 najgora i na njihovo mesto stavi djecu
        population=[child1,child2]+[individual for individual, _ in sorted_population[2:]] 


    return overall_best_string,overall_best_fitness
    
##AAAAAAA fixed it sad mi za kraj cuva najbolje <333



 
        
        

In [ ]:
if __name__ == "__main__":
    string_length = int(input("unesi broj karaktera u stringu"))
    population_size = int(input("unesi population size"))
    mutation_rate = 0.2
    max_consecutive_zeros = int(input("unesi max broj uzastopnih nula"))
    best_string, best_fitness = genetic_algorithm(string_length, population_size, mutation_rate, max_consecutive_zeros)
    print(f"Best string: {best_string}, Best fitness: {best_fitness}")
